In [2]:
import requests
import fastavro
from fastavro.schema import load_schema

# Your API key
api_key = '40cff5d3f93c059e8b97afe8fdc7327d'

# Base URL for Scopus API
base_url = 'https://api.elsevier.com/content/search/scopus'

# Broad search query (broad term like 'research')
query = 'PUBYEAR < 2024 AND DOCTYPE("ar") OR DOCTYPE(cp)'  # Broad search term

# Headers with your API key
headers = {
    'X-ELS-APIKey': api_key
}

# Parameters for the API request (e.g., pagination settings)
count = 25  # Fetch 25 papers per request
total_papers = 0
start = 0  # Starting point for pagination
papers = []  # To store the collected papers
entriesss = []
abstractsss = []
prism_urls = []
datata =[]

# Loop to fetch papers until we reach 100
while total_papers < 100:
    params = {
        'query': query,
        'count': count,
        'start': start
    }
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        datata.append(data)
        entries = data.get('search-results', {}).get('entry', [])
        entriesss.append(entries)
        if not entries:
            break
        for entry in entries:
            title = entry.get('dc:title', 'N/A')
            authors = entry.get('dc:creator', 'N/A')
            abstract = entry.get('dc:description', None) or entry.get('dc:absract', None)
            classifications = entry.get('classification', [])
            date_publication = entry.get('prism:coverDate', '').split('-')
            affiliations = entry.get('affiliation', [])
            references = entry.get('citedby-count', [])
            keywords = entry.get('authkeywords', [])
            prism_url = entry.get('prism:url', [])

            prism_urls.append(prism_url)

            paper = {
                "title": title,
                "abstract": abstract,
                "classifications": classifications,
                "date_publication": {
                    "year": date_publication[0],
                    "month": date_publication[1],
                    "day": date_publication[2]
                },
                "affiliation": [{
                    "affiliation-city": affiliation.get('affiliation-city', None),
                    "@id": affiliation.get('@id', ''),
                    "affilname": affiliation.get('affilname', ''),
                    "@href": affiliation.get('@href', ''),
                    "affiliation-country": affiliation.get('affiliation-country', None)
                } for affiliation in affiliations],
                "references": references,
                "keywords": keywords,
            }
            papers.append(paper)
            total_papers += 1
            if total_papers >= 100:
                break
        start += count
    else:
        print(f"Error: {response.status_code}")
        break

# Define your schema
schema = load_schema('../scopus.avsc')

# Write data to Avro file
with open('additional_scopus_data.avro', 'wb') as out_file:
    fastavro.writer(out_file, schema, papers)

ModuleNotFoundError: No module named 'fastavro'

In [ ]:
# allkeys they provide is all features that we can get from the scopus api
entriesss[0][0].keys()

dict_keys(['@_fa', 'link', 'prism:url', 'dc:identifier', 'eid', 'dc:title', 'prism:publicationName', 'prism:issn', 'prism:volume', 'prism:issueIdentifier', 'prism:pageRange', 'prism:coverDate', 'prism:coverDisplayDate', 'prism:doi', 'citedby-count', 'prism:aggregationType', 'subtype', 'subtypeDescription', 'source-id', 'openaccess', 'openaccessFlag'])

In [ ]:
# Path to the Avro file
avro_file_path = 'additional_scopus_data.avro'

# Open and read the Avro file
with open(avro_file_path, 'rb') as f:
    # Read the Avro records
    reader = fastavro.reader(f)
    
    # Print out the records from the Avro file
    for record in reader:
        print(record)

{'title': 'Minutes', 'abstract': None, 'classifications': [], 'date_publication': {'year': '2023', 'month': '12', 'day': '31'}, 'affiliation': [], 'references': ['0'], 'keywords': []}
{'title': '(IM)PIETY: ISLAM, PIOUS SINNING AND REGULATED TRANSGRESSIONS', 'abstract': None, 'classifications': [], 'date_publication': {'year': '2023', 'month': '12', 'day': '31'}, 'affiliation': [{'affiliation-city': 'Christchurch', '@id': '', 'affilname': 'University of Canterbury', '@href': '', 'affiliation-country': 'New Zealand'}], 'references': ['0'], 'keywords': []}
{'title': 'KNOWLEDGE OF THE MILKY WAY IN THE ARABIC CULTURAL REGION BETWEEN THE 8<sup>TH</sup> AND 15<sup>TH</sup> CENTURIES', 'abstract': None, 'classifications': [], 'date_publication': {'year': '2023', 'month': '12', 'day': '31'}, 'affiliation': [{'affiliation-city': 'Koln', '@id': '', 'affilname': 'Universität zu Köln', '@href': '', 'affiliation-country': 'Germany'}], 'references': ['0'], 'keywords': []}
{'title': 'WEAVING MODERNITY

In [ ]:
avro_file_path = 'additional_scopus_data.avro'

# Function to count the number of rows in an Avro file
def count_rows_in_avro(file_path):
    with open(file_path, 'rb') as f:
        reader = fastavro.reader(f)
        row_count = sum(1 for _ in reader)
    return row_count

# Count the rows in the Avro file
row_count = count_rows_in_avro(avro_file_path)
print(f"Number of rows in the Avro file: {row_count}")

Number of rows in the Avro file: 100
